In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

df = pd.DataFrame(np.random.randint(0, int(1e8), (10000, 1000)))

# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Now you can use `progress_apply` instead of `apply`
df.groupby(0).progress_apply(lambda x: x**2)

100%|██████████████████████████████████| 10000/10000 [00:07<00:00, 1370.29it/s]


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-113040384,-491048316,1632275520,901433881,705482148,476818153,985071857,-147046044,-1237815487,802099200,...,1644923665,671321360,-1416514431,78612809,236031425,925033616,-646670303,1056328913,250495953,-1594752639
1,1197573609,-2142031808,1309328577,-931520535,1463888457,-91003904,-1094130108,1522663680,1624244292,-239741424,...,2144720057,821992609,958727121,749520465,915830820,-475023484,-893327984,1071648768,-980364767,821666704
2,-405144156,-1190619487,-1256944368,-241281511,1019075044,-1700732892,82388880,860125377,547525284,491697369,...,-1191434247,-1673738719,399686116,-1810330172,-771763383,1792520425,-2090713088,531302417,740508129,460131392
3,453291609,1841691753,-1651335831,681880273,1695942865,1773069312,-928775719,-408320423,799037337,-594876479,...,513889668,868725824,64729521,2052181937,1489642593,1634917700,-1077491456,-1178520031,-1037713436,1283210816
4,1121815241,-413273431,-2095533695,-778886384,-1864381680,186010977,-1531927311,790831937,1910986057,-1130794551,...,1641098276,-646913151,34296121,-211313031,1901932161,1031648057,-1966223503,1418833737,-1946527612,320563108
5,-1989357808,-1090526208,258900977,784975817,-1940769143,-1998536304,-862431168,-184002431,-1924661959,-370295695,...,-829981855,-1261658535,368099584,-2041430684,-1120378351,1097449889,1402845796,-1884992591,-1710689287,-874684351
6,122175849,-1808976624,-1610601372,1470530889,557956609,918896481,566736073,313041508,1941506624,856714521,...,2016981401,-2032641180,-35032000,-950819840,1693647785,-755393031,1101774313,1921191936,1638458289,635722852
7,-1300991,-657326815,1717784737,515160873,1963400321,-699673039,1620883908,-1939380508,781307908,1210685956,...,-1819961015,-904403135,78415193,-514102208,814743552,1385976449,-385972735,1151628153,416252420,-541463487
8,-1551526743,573426340,459121337,-1241032303,393755377,1312266753,-1695103791,-910508671,-286832496,-1743382012,...,-751993487,-404591559,-772844511,1268008512,-1610588784,-1368637296,-632834711,-1227493863,-461201423,1925197417
9,81201552,-196401628,1135377028,-2104547343,567792113,-981201264,443596801,717801281,-773387580,713843377,...,-1589961756,1196312848,1826246065,-1244421855,-367040735,178064369,-1207165424,153932649,-836854527,1818103737


In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [28]:
letters = ['A', 'B', 'D', 'E', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'Sade', 'Shin', 'T', 'Tet', 'U', 'W', 'Y', 'Z']
ur3 = []
for l in letters:
    response = requests.get("http://altorient.gko.uni-leipzig.de/SAD/" + l + ".htm")
    doc = response.content
    new_doc = UnicodeDammit.detwingle(doc)
    soup = BeautifulSoup(new_doc, 'html.parser')
    for n in soup.find_all("p"):
        if "Ur III" in n.get_text(): 
            ur3.append(n.get_text())

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [29]:
ur3

['Ur III ezem a-bu-um-ma-Å¡è\nCUSAS 4, 52 â€œfor the Abum festivalâ€�.',
 'agammu â€œmarsh(es), lagoonâ€�; Ur III, SB',
 'Ur III i7 a-ga-mu-um CUSAS 4, 652.',
 'agarinnu â€œwomb;\nbasin for mixing mortarâ€�; + Ur III',
 '1. Ur III im a-ga-ri-nu-um ka-la(/lá)-ka gub-ba CUSAS 3,\n7: 32â€™; 8: 25 â€œemployed at the excavation for mixing earth in a basinâ€�, s.\nHeimpel, CUSAS 5, 239f.; cf. Sallaberger, CUSAS 6, 359.',
 'agÃ» I â€œtiaraâ€�; Ur III: â€œa\nspecification of textiles(?)â€� ',
 'Ur III\nGADA.GEN a-gi4-um CUSAS 3, 573: 1 â€œordinary a. linen textileâ€�; tÃºgḫa-um a-gi4-um PN\nCUSAS 3, 581: 1; tÃºgNÍG.LÃ�M 3-kam-Ãºs\na-gi4-um â€œ3rd quality a. N.-textileâ€�. S. CUSAS 4, 2 for\nfurther ref. Sallaberger, CUSAS 6, 357 postulates a meaning â€œdyed, coloredâ€�.',
 'Ur III tÃºgá-gu4-ḫu-um pi lugal\nCUSAS 3, 578: 1, 579: 1, 591: 1 â€œroyal PI-beltâ€�; tÃºgá-[gu4-ḫu-um] Ãºs 3-[kam-Ãºs]\nCUSAS 3, 759: 1 â€œ3rd quality beltâ€�. S. also CUSAS 6, p. 258f., 1582:\n8, 30.',
 '1. Ur III 20 in-d

In [35]:
ur3_2 = []
for i in ur3:
    i = i.replace('â€�', '"')
    i = i.replace('â€œ', '"')
    i = i.replace('tÃºg', '{tug₂}')
    i = i.replace('\n', ' ')
    i = i.replace('Ãº', 'ú')
    ur3_2.append(i)

In [36]:
ur3_2

['Ur III ezem a-bu-um-ma-Å¡è CUSAS 4, 52 "for the Abum festival".',
 'agammu "marsh(es), lagoon"; Ur III, SB',
 'Ur III i7 a-ga-mu-um CUSAS 4, 652.',
 'agarinnu "womb; basin for mixing mortar"; + Ur III',
 '1. Ur III im a-ga-ri-nu-um ka-la(/lá)-ka gub-ba CUSAS 3, 7: 32â€™; 8: 25 "employed at the excavation for mixing earth in a basin", s. Heimpel, CUSAS 5, 239f.; cf. Sallaberger, CUSAS 6, 359.',
 'agÃ» I "tiara"; Ur III: "a specification of textiles(?)" ',
 'Ur III GADA.GEN a-gi4-um CUSAS 3, 573: 1 "ordinary a. linen textile"; {tug₂}ḫa-um a-gi4-um PN CUSAS 3, 581: 1; {tug₂}NÍG.LÃ�M 3-kam-ús a-gi4-um "3rd quality a. N.-textile". S. CUSAS 4, 2 for further ref. Sallaberger, CUSAS 6, 357 postulates a meaning "dyed, colored".',
 'Ur III {tug₂}á-gu4-ḫu-um pi lugal CUSAS 3, 578: 1, 579: 1, 591: 1 "royal PI-belt"; {tug₂}á-[gu4-ḫu-um] ús 3-[kam-ús] CUSAS 3, 759: 1 "3rd quality belt". S. also CUSAS 6, p. 258f., 1582: 8, 30.',
 '1. Ur III 20 in-daḫ-Å¡u-um MVN 6, 59: 2; cf. AUCT 1, 974: 2. S. CAD 

In [17]:
new_doc.decode("utf-8")

'☃☃☃“I like snowmen!”'